In [3]:
import json
import boto3
import pandas as pd

from utils.analysis_utils import classify_one_feedback
from utils.database import get_field_value
from pymongo import MongoClient


In [4]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING)

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

# Generating Elementary Subjects

After creating all the extractions, our objective is to generate elementary subjects.

We use the following method:

1. For each extraction, we attempt to classify it among the existing elementary subjects.
2. If no subject matches, we generate a new one.
3. As we progress, we also verify that we are not generating duplicates.

## Process

We classify and generate our elementary subjects progressively:

1. Classify each extraction
2. Generate new subjects as needed
3. Check for duplicates

By the end of this process, after classifying everything, we will have generated all our subjects.

## Database Storage

The elementary subjects are pushed into the `elementary_subjects_dev` database.

In [5]:
BRAND = 'columbuscafe_test'

BRAND_DESCR = '''
Feedbacks are from client of columbus cafe.
Columbus Café & Co est une chaîne française de cafés.
'''

In [25]:
# getting feedbacks from mongo

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

(2994, 16)

In [9]:
# keeping only feedbacks with extractions and splitted_analysis_v2

subdf = from_mongo[from_mongo['extractions'].notna()]
subdf = subdf[subdf['splitted_analysis_v2'].notna()]
print(subdf.shape)

row = subdf.sample().iloc[0]

print(row.verbatim['text'])

(20, 16)
Demandé un chocolat chaud il est tiède. De olus aucune tranquillité avec le télétravail c'est très désagréable d'entendre la conversation des uns et des autres. A revoir


In [12]:
extractions = get_field_value(
    feedback_id=row['_id'],
    field_name='extractions',
)

extractions

[{'sentiment': 'POSITIVE',
  'extraction': 'Établissement très propre',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Personnel très aimable',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'},
 {'sentiment': 'POSITIVE',
  'extraction': 'Recommandation à 100%',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'}]

The function `classify_one_feedback` is used to classify each extraction.

In [14]:
res = classify_one_feedback(
    feedback_id=row['_id'],
    extractions=extractions,
    model="gpt-4o-mini",
    brand=BRAND,
    brand_descr=BRAND_DESCR,
    language="french",
    extractions_column="extractions",
    update_mongo=True
)

res

Updating feedback columbuscafe_test/a965ce53ffc4263dd876 with updates: {'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Établissement très propre', 'text': 'Très propre le personnel est très aimable je vous le conseille à 100%', 'elementary_subjects': ["Ambiance : Propreté de l'établissement"], 'topics': []}, {'sentiment': 'POSITIVE', 'extraction': 'Personnel très aimable', 'text': 'Très propre le personnel est très aimable je vous le conseille à 100%', 'elementary_subjects': ["Service : Qualité de l'équipe"], 'topics': []}, {'sentiment': 'POSITIVE', 'extraction': 'Recommandation à 100%', 'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'}], 'splitted_analysis_v2': [{'text': 'Très propre le personnel est très aimable je vous le conseille à 100%', 'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Établissement très propre', 'text': 'Très propre le personnel est très aimable je vous le conseille à 100%', 'elementary_subjects': ["Ambiance : Pro

[{'sentiment': 'POSITIVE',
  'extraction': 'Établissement très propre',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%',
  'elementary_subjects': ["Ambiance : Propreté de l'établissement"],
  'topics': []},
 {'sentiment': 'POSITIVE',
  'extraction': 'Personnel très aimable',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%',
  'elementary_subjects': ["Service : Qualité de l'équipe"],
  'topics': []},
 {'sentiment': 'POSITIVE',
  'extraction': 'Recommandation à 100%',
  'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'}]

In [1]:
# this function allows to parallelize the analysis process and to save the results on the mongo database
from utils.analysis_utils import run_analysis_full_parallel

In [13]:
to_analyse = subdf.sample(10) # test on a subset
print('To extract:', to_analyse.shape[0])

extractions_with_ids = to_analyse[['extractions', '_id']].to_dict(orient='records')

To extract: 10


In [ ]:
analysis = run_analysis_full_parallel(
    extractions_with_ids,
    BRAND,
    brand_descr=BRAND_DESCR,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

analysis

In [16]:
analysis

[{'id': 'columbuscafe_test/13892e2103825cf06d02',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Appréciation générale',
    'text': '1👍'}]},
 {'id': 'columbuscafe_test/a965ce53ffc4263dd876',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Établissement très propre',
    'text': 'Très propre le personnel est très aimable je vous le conseille à 100%',
    'elementary_subjects': ["Ambiance : Propreté de l'établissement"],
    'topics': []},
   {'sentiment': 'POSITIVE',
    'extraction': 'Personnel très aimable',
    'text': 'Très propre le personnel est très aimable je vous le conseille à 100%',
    'elementary_subjects': ["Service : Qualité de l'équipe"],
    'topics': []},
   {'sentiment': 'POSITIVE',
    'extraction': 'Recommandation à 100%',
    'text': 'Très propre le personnel est très aimable je vous le conseille à 100%'}]},
 {'id': 'columbuscafe_test/09849c25a5224e258d60',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Très bon accue

## Check elementary subjects that we generated

In [17]:
from utils.database import get_elementary_subjects

In [19]:
positive_elementary_subjects = get_elementary_subjects(BRAND, 'positive')

len(positive_elementary_subjects)

18

In [20]:
[x['elementary_subject'] for x in positive_elementary_subjects[:5]]

['Ambiance : Ambiance agréable',
 'Service : Accueil chaleureux',
 'Service : Problemlösung : Schnelle Problemlösung bei Fehlern',
 'Produits : Qualité des produits : Qualité de la nourriture',
 'Produits : Qualité des produits : Qualité du café']

In [24]:
negative_elementary_subjects = get_elementary_subjects(BRAND, 'negative')

print(len(negative_elementary_subjects))

[x['elementary_subject'] for x in negative_elementary_subjects[:5]]

11


['Service : Comportement du personnel : Serveur impoli ou nerveux',
 'Magasin : Aménagement : Emplacement du mobilier inapproprié',
 "Accessibilité : Difficulté d'accès pour les fauteuils roulants",
 'Produits : Problème de qualité : Chocolat non consommable',
 'Prix : Tarifs élevés']